In [1]:
!pip install imbalanced-learn > /dev/null 2>&1

In [2]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import tensorflow as tf

from joblib import dump, load

from imblearn.over_sampling import SMOTE

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

## 0.03

In [3]:
train = pd.read_csv("/notebooks/FinalDataset/top003_train_encoded.csv")
test = pd.read_csv("/notebooks/FinalDataset/top003_test_encoded.csv")

x_train = train.drop('attack_cat', axis=1)
y_train = train['attack_cat']

x_test = test.drop('attack_cat', axis=1)
y_test = test['attack_cat']

del train, test

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(123504, 22)
(123504,)
(41169, 22)
(41169,)


### Unbalanced

In [4]:
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_003_Unbalanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_003_Unbalanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

5    35.720301
3    27.098717
4    14.728268
2     9.930852
6     8.494462
0     1.629097
1     1.413719
7     0.887421
8     0.097163
Name: attack_cat, dtype: float64
Epoch 1/5000
3860/3860 [==============================] - 13s 3ms/step - loss: 0.9045 - accuracy: 0.6567 - val_loss: 0.8483 - val_accuracy: 0.6726
Epoch 2/5000
3860/3860 [==============================] - 11s 3ms/step - loss: 0.8230 - accuracy: 0.6980 - val_loss: 0.8033 - val_accuracy: 0.7077
Epoch 3/5000
3860/3860 [==============================] - 11s 3ms/step - loss: 0.7896 - accuracy: 0.7121 - val_loss: 0.7786 - val_accuracy: 0.7286
Epoch 4/5000
3860/3860 [==============================] - 11s 3ms/step - loss: 0.7708 - accuracy: 0.7192 - val_loss: 0.7728 - val_accuracy: 0.7175
Epoch 5/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.7602 - accuracy: 0.7235 - val_loss: 0.7556 - val_accuracy: 0.7276
Epoch 6/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.7513 - accuracy:

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Balanced

In [5]:
oversample = SMOTE(sampling_strategy='not majority', random_state=42)
x_train, y_train = oversample.fit_resample(x_train, y_train)
x_train, y_train = shuffle(x_train, y_train, random_state=42)

y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_003_Balanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_003_Balanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

1    11.111111
5    11.111111
0    11.111111
6    11.111111
7    11.111111
8    11.111111
2    11.111111
4    11.111111
3    11.111111
Name: attack_cat, dtype: float64
Epoch 1/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.3795 - accuracy: 0.4674 - val_loss: 1.1705 - val_accuracy: 0.5312
Epoch 2/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.2507 - accuracy: 0.5067 - val_loss: 1.0478 - val_accuracy: 0.6283
Epoch 3/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.1879 - accuracy: 0.5270 - val_loss: 1.0086 - val_accuracy: 0.6273
Epoch 4/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 1.1591 - accuracy: 0.5375 - val_loss: 0.9957 - val_accuracy: 0.6364
Epoch 5/5000
12408/12408 [==============================] - 35s 3ms/step - loss: 1.1379 - accuracy: 0.5462 - val_loss: 0.9813 - val_accuracy: 0.6136
Epoch 6/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.1223

## 0.02

In [6]:
train = pd.read_csv("/notebooks/FinalDataset/top002_train_encoded.csv")
test = pd.read_csv("/notebooks/FinalDataset/top002_test_encoded.csv")

x_train = train.drop('attack_cat', axis=1)
y_train = train['attack_cat']

x_test = test.drop('attack_cat', axis=1)
y_test = test['attack_cat']

del train, test

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(123504, 154)
(123504,)
(41169, 154)
(41169,)


## Unbalanced

In [7]:
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_002_Unbalanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_002_Unbalanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

5    35.720301
3    27.098717
4    14.728268
2     9.930852
6     8.494462
0     1.629097
1     1.413719
7     0.887421
8     0.097163
Name: attack_cat, dtype: float64
Epoch 1/5000
3860/3860 [==============================] - 13s 3ms/step - loss: 0.7335 - accuracy: 0.7328 - val_loss: 0.6741 - val_accuracy: 0.7443
Epoch 2/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6660 - accuracy: 0.7532 - val_loss: 0.6526 - val_accuracy: 0.7611
Epoch 3/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6496 - accuracy: 0.7589 - val_loss: 0.6429 - val_accuracy: 0.7631
Epoch 4/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6408 - accuracy: 0.7612 - val_loss: 0.6415 - val_accuracy: 0.7626
Epoch 5/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6306 - accuracy: 0.7646 - val_loss: 0.6411 - val_accuracy: 0.7637
Epoch 6/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6239 - accuracy:

### Balanced

In [8]:
oversample = SMOTE(sampling_strategy='not majority', random_state=42)
x_train, y_train = oversample.fit_resample(x_train, y_train)
x_train, y_train = shuffle(x_train, y_train, random_state=42)

y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_002_Balanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_002_Balanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

1    11.111111
5    11.111111
0    11.111111
6    11.111111
7    11.111111
8    11.111111
2    11.111111
4    11.111111
3    11.111111
Name: attack_cat, dtype: float64
Epoch 1/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.0825 - accuracy: 0.5660 - val_loss: 0.8731 - val_accuracy: 0.6537
Epoch 2/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 0.9637 - accuracy: 0.6103 - val_loss: 0.8010 - val_accuracy: 0.6965
Epoch 3/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 0.9210 - accuracy: 0.6292 - val_loss: 0.8018 - val_accuracy: 0.6893
Epoch 4/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.8916 - accuracy: 0.6437 - val_loss: 0.7847 - val_accuracy: 0.7173
Epoch 5/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.8727 - accuracy: 0.6522 - val_loss: 0.8005 - val_accuracy: 0.6992
Epoch 6/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.8614

## 0.01

In [9]:
train = pd.read_csv("/notebooks/FinalDataset/top001_train_encoded.csv")
test = pd.read_csv("/notebooks/FinalDataset/top001_test_encoded.csv")

x_train = train.drop('attack_cat', axis=1)
y_train = train['attack_cat']

x_test = test.drop('attack_cat', axis=1)
y_test = test['attack_cat']

del train, test

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(123504, 168)
(123504,)
(41169, 168)
(41169,)


### Unbalanced

In [10]:
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_001_Unbalanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_001_Unbalanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

5    35.720301
3    27.098717
4    14.728268
2     9.930852
6     8.494462
0     1.629097
1     1.413719
7     0.887421
8     0.097163
Name: attack_cat, dtype: float64
Epoch 1/5000
3860/3860 [==============================] - 13s 3ms/step - loss: 0.7418 - accuracy: 0.7299 - val_loss: 0.6668 - val_accuracy: 0.7574
Epoch 2/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6642 - accuracy: 0.7547 - val_loss: 0.6558 - val_accuracy: 0.7583
Epoch 3/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6490 - accuracy: 0.7595 - val_loss: 0.6467 - val_accuracy: 0.7604
Epoch 4/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6387 - accuracy: 0.7621 - val_loss: 0.6473 - val_accuracy: 0.7598
Epoch 5/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6308 - accuracy: 0.7644 - val_loss: 0.6351 - val_accuracy: 0.7626
Epoch 6/5000
3860/3860 [==============================] - 12s 3ms/step - loss: 0.6230 - accuracy:

### Balanced

In [11]:
oversample = SMOTE(sampling_strategy='not majority', random_state=42)
x_train, y_train = oversample.fit_resample(x_train, y_train)
x_train, y_train = shuffle(x_train, y_train, random_state=42)

y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

proportions = y_train.value_counts(normalize=True) * 100
print(proportions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_ohe.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=5000, callbacks=[early_stopping])
model.save('/notebooks/Models/MLP/MLP_001_Balanced.h5')

model = load_model("/notebooks/Models/MLP/MLP_001_Balanced.h5")
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

1    11.111111
5    11.111111
0    11.111111
6    11.111111
7    11.111111
8    11.111111
2    11.111111
4    11.111111
3    11.111111
Name: attack_cat, dtype: float64
Epoch 1/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 1.0960 - accuracy: 0.5621 - val_loss: 0.8978 - val_accuracy: 0.6531
Epoch 2/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 0.9709 - accuracy: 0.6084 - val_loss: 0.8871 - val_accuracy: 0.6180
Epoch 3/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.9212 - accuracy: 0.6308 - val_loss: 0.8507 - val_accuracy: 0.6741
Epoch 4/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.8922 - accuracy: 0.6421 - val_loss: 0.8165 - val_accuracy: 0.6808
Epoch 5/5000
12408/12408 [==============================] - 34s 3ms/step - loss: 0.8733 - accuracy: 0.6513 - val_loss: 0.8269 - val_accuracy: 0.6918
Epoch 6/5000
12408/12408 [==============================] - 33s 3ms/step - loss: 0.8594